In [63]:
import math
import numpy as np
import sys
from typing import List, Tuple
from functools import lru_cache, reduce, cmp_to_key
from itertools import (
    starmap,
    accumulate,
    chain,
    islice,
    product,
    tee,
    pairwise,
    combinations,
)
from collections import Counter, deque
from heapq import heapify, heappushpop, heappop, heappush
from operator import add, neg
from bisect import bisect_right
import matplotlib.pyplot as plt

import arrays
import trees
import lists
import graphs
import search
from importlib import reload

reload(arrays)
reload(trees)
reload(lists)
reload(graphs)
reload(search)
import arrays as ar
import trees as tr
import lists as ll
import graphs as g
from lists import Node

print()

In [72]:
def fix_bst(root):
    a = b = None

    def rec(p, n):
        nonlocal a, b
        if not n:
            return p
        p = rec(p, n.left)
        if p and p.data > n.data:
            a, b = a or p, n
        return rec(n, n.right)

    rec(None, root)

    if a:
        a.data, b.data = b.data, a.data

In [270]:
def closest_palindrome_number(n: int) -> int:
    "Return the closest number to `n` that is a palindrome number."

    def cmp(a, b):
        s = str(b)
        m = len(s) // 2
        c = int("".join(s[i if i < m else -(i + 1)] for i in range(len(s))))
        return c if abs(c - n) < abs(a - n) else a

    d = n > 9 and 10 ** int((math.log(n, 10) + 1) // 2)
    return reduce(lambda a, f: cmp(a, n + f), (-d, -1, 0, d), 0) if d else n

In [272]:
closest_palindrome_number(100)

-1
-10
0
10


99

In [244]:
int(math.log(5, 10))

0